# Routing

- Author: [Lee Jungbin](https://github.com/leebeanbin)
- Peer Review: [Teddy Lee](https://github.com/teddylee777), [Musang Kim](https://github.com/musangk)
- Proofread:
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/05-Memory/06-ConversationSummaryMemory.ipynb) [![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/05-Memory/06-ConversationSummaryMemory.ipynb)

## Overview

This tutorial introduces key concepts and practical applications of `RunnableSequence` in LangChain. RunnableSequence is a powerful tool for creating sequential processing pipelines that enable structured data flow and transformation.

Key features of RunnableSequence include:
- Sequential execution of multiple operations
- Automatic data passing between steps
- Pipeline-level error handling
- Integration with other Runnable components
- Support for async operations

Through detailed examples and best practices, you'll learn how to effectively use RunnableSequence to build robust and maintainable data processing pipelines.

### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [Understanding RunnableSequence](#understanding-runnablesequence)
- [Basic Usage](#basic-usage)
- [Advanced Features](#advanced-features)
- [Best Practices](#best-practices)
- [Real-world Examples](#real-world-examples)
- [Performance Optimization](#performance-optimization)

### References
- [RunnableSequence API Reference](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableSequence.html)
- [LangChain Expression Language (LCEL)](https://python.langchain.com/docs/expression_language/interface)
---

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

[Note]
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can check out the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [16]:
%%capture --no-stderr
!pip install langchain-opentutorial
!pip install langchain-openai

In [17]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_openai",
        "pydantic",
    ],
    verbose=False,
    upgrade=False,
)

You can alternatively set `OPENAI_API_KEY` in `.env` file and load it. 

[Note] This is not necessary if you've already set `OPENAI_API_KEY` in previous steps.

In [3]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "04-Routing",
    }
)

Environment variables have been set successfully.


In [4]:
# Load environment variables
# Reload any variables that need to be overwritten from the previous cell

from dotenv import load_dotenv

load_dotenv(override=True)

True

## Understanding RunnableSequence

`RunnableSequence` is a fundamental component in LangChain that enables the creation of sequential processing pipelines. It allows developers to chain multiple operations together where the output of one step becomes the input of the next step.

### Key Concepts

1. **Sequential Processing**
   - Ordered execution of operations
   - Automatic data flow between steps
   - Clear pipeline structure

2. **Data Transformation**
   - Input preprocessing
   - State management
   - Output formatting

3. **Error Handling**
   - Pipeline-level error management
   - Step-specific error recovery
   - Fallback mechanisms

Let's explore these concepts with practical examples.

### Simple Example

First, we will create a Chain that classifies incoming questions into one of three categories: math, science, or other.

In [18]:
from langchain_core.runnables import RunnableSequence
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

# Basic Example: Text Processing Pipeline
basic_chain = (
    # Step 1: Input handling and prompt creation
    PromptTemplate.from_template("Summarize this text in three sentences: {text}")
    # Step 2: LLM processing
    | ChatOpenAI(temperature=0)
    # Step 3: Output parsing
    | StrOutputParser()
)

# Example usage
result = basic_chain.invoke({"text": "This is a sample text to process."})
print(result)

This text is a sample for processing purposes. It is likely being used as an example for a specific task or function. The content of the text is not specified beyond being a sample.


## Basic Usage

RunnableSequence can be used in various ways, from simple text processing to complex data transformations. Here are some fundamental usage patterns.

### 1. Simple Text Processing

In [19]:
from langchain_core.runnables import RunnableSequence
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

"""
Basic Text Generation Pipeline
- Purpose: Generate and process AI-related technical content
- Features: Text generation, processing, and summarization
- Use case: Creating technical AI documentation or explanations
"""

# Step 1: Define the text generation chain
text_generation_chain = (
    # Create prompt template for AI content generation
    PromptTemplate.from_template(
        """Generate a detailed technical explanation about {topic} in AI/ML field.
            Include:
            - Core technical concepts
            - Implementation details
            - Real-world applications
            - Technical challenges
            """
    )
    # Process with LLM using moderate temperature for creativity
    | ChatOpenAI(temperature=0.7)
    # Convert output to clean string
    | StrOutputParser()
)

# Example usage
result = text_generation_chain.invoke({"topic": "Transformer architecture in LLMs"})
print("Generated Content:", result)

Generated Content: Transformer architecture in Large Language Models (LLMs) is a crucial advancement in the field of artificial intelligence and machine learning. This architecture has revolutionized natural language processing tasks by enabling models to learn long-range dependencies and capture complex patterns in sequential data without the need for recurrent neural networks.

Core technical concepts:
The key technical concept in the Transformer architecture is the self-attention mechanism, which allows the model to weigh the importance of different input tokens when making predictions. This mechanism enables the model to process input sequences in parallel, making it more efficient than recurrent neural networks. The Transformer architecture consists of multiple layers of self-attention and feedforward neural networks, allowing the model to capture intricate relationships within the input data.

Implementation details:
The Transformer architecture is typically implemented using a s

## Advanced Features

### 1. Structured Content Analysis Pipeline

In [20]:
"""
Content Analysis Pipeline
- Purpose: Analyze the generated AI technical content
- Features: Structured summarization and key point extraction
- Use case: Understanding and organizing technical documentation
"""

# Step 2: Create the analysis chain
analysis_chain = (
    # Create prompt template for content analysis
    PromptTemplate.from_template(
        """Analyze this technical AI content and provide a structured breakdown:
            
            Content to analyze:
            {generated_content}
            
            Provide analysis in the following format:
            1. Core Technical Concepts:
               - List the main technical principles
               - Explain key theoretical foundations
            
            2. Implementation Insights:
               - Describe the architecture details
               - Highlight critical components
            
            3. Practical Applications:
               - Identify real-world use cases
               - Note implementation considerations
            
            4. Technical Challenges:
               - List potential difficulties
               - Suggest mitigation strategies
            """
    )
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
)

# Create combined pipeline
combined_chain = (
    # Step 1: Generate technical content
    {"generated_content": lambda x: text_generation_chain.invoke({"topic": x["topic"]})}
    # Step 2: Analyze the generated content
    | analysis_chain
)

# Example usage
result = combined_chain.invoke({"topic": "Transformer architecture in LLMs"})
print("\nAnalysis of the Content:", result)


Analysis of the Content: 1. Core Technical Concepts:
   - The main technical principle is the self-attention mechanism in the Transformer architecture.
   - The key theoretical foundation is the ability of the model to focus on different parts of the input sequence by assigning weights to each input token based on its relevance to other tokens.

2. Implementation Insights:
   - The Transformer architecture consists of an encoder-decoder structure with multiple layers of self-attention mechanisms and feed-forward neural networks.
   - Each layer includes a multi-head self-attention mechanism to attend to different parts of the input sequence simultaneously, and a feed-forward neural network to capture non-linear relationships in the data.

3. Practical Applications:
   - Real-world applications of Transformer architecture include natural language processing tasks such as machine translation, text summarization, and language modeling.
   - Transformer-based models like BERT, GPT, and T5

In [24]:
"""
Structured Technical Evaluation Pipeline
- Purpose: Detailed technical evaluation of AI content with specific metrics
- Features: Metric-based analysis, structured output format
- Use case: Technical documentation and evaluation reports
"""

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnableParallel
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List


# Define Pydantic models
class TechnicalDepth(BaseModel):
    complexity_score: float = Field(description="Technical complexity score (1-10)")
    depth_analysis: str = Field(description="Analysis of technical depth")


class ImplementationEvaluation(BaseModel):
    feasibility: str = Field(description="Implementation feasibility assessment")
    required_resources: List[str] = Field(
        description="List of required technical resources"
    )


class ImpactAssessment(BaseModel):
    industry_impact: str = Field(description="Potential impact on industry")
    technical_advantages: List[str] = Field(description="Key technical advantages")


class EvaluationOutput(BaseModel):
    technical_depth: TechnicalDepth
    implementation_evaluation: ImplementationEvaluation
    impact_assessment: ImpactAssessment


# Text generation chain
text_generation_chain = PromptTemplate.from_template(
    "Write a detailed technical explanation about {topic}."
) | ChatOpenAI(temperature=0.7)

# Analysis chain
analysis_chain = PromptTemplate.from_template(
    "Analyze the following technical content:\n\n{generated_content}"
) | ChatOpenAI(temperature=0)

# Evaluation chain
evaluation_parser = JsonOutputParser(pydantic_object=EvaluationOutput)

evaluation_chain = (
    PromptTemplate.from_template(
        """Perform a detailed technical evaluation of the following AI content:
        
        {generated_content}
        
        Provide a structured evaluation following these criteria:
        1. Technical Depth Assessment
        2. Implementation Feasibility
        3. Industry Impact Analysis
        
        {format_instructions}
        """
    )
    | ChatOpenAI(temperature=0)
    | evaluation_parser
)

# Create comprehensive pipeline
comprehensive_chain = RunnableParallel(
    {
        "content": lambda x: text_generation_chain.invoke({"topic": x["topic"]}),
        "analysis": lambda x: analysis_chain.invoke(
            {"generated_content": text_generation_chain.invoke({"topic": x["topic"]})}
        ),
        "evaluation": lambda x: evaluation_chain.invoke(
            {
                "generated_content": text_generation_chain.invoke(
                    {"topic": x["topic"]}
                ),
                "format_instructions": evaluation_parser.get_format_instructions(),
            }
        ),
    }
)

# Example usage
result = comprehensive_chain.invoke({"topic": "Transformer architecture in LLMs"})
print("\nComprehensive Analysis:")
print("\nOriginal Content:", result["content"])
print("\nGeneral Analysis:", result["analysis"])
print("\nTechnical Evaluation:", result["evaluation"])


Comprehensive Analysis:

Original Content: content='Transformer architecture in Large Language Models (LLMs) is a type of neural network architecture that has been widely adopted in natural language processing tasks due to its ability to capture long-range dependencies and relationships in sequential data. The Transformer architecture was first introduced in the paper "Attention is All You Need" by Vaswani et al. in 2017, and has since become a key component of state-of-the-art language models such as BERT, GPT, and T5.\n\nThe core idea behind the Transformer architecture is the use of self-attention mechanisms to compute representations of input sequences. Self-attention allows the model to weigh the importance of each token in the input sequence when generating a representation for a given token. This enables the model to focus on relevant parts of the input sequence and capture dependencies between tokens that are far apart in the sequence.\n\nThe Transformer architecture consists 